In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:00


In [2]:
import pennylane as qml
import pennylane.numpy as np

To implement this evolution, it is necessary to find the $U(t)$ for the Hamiltonian as combinations of quantum gates. In this way, we derived the following expression for $U(t)$:

$$
U(t) \approx \prod_{l=1}^n C_{\text{not}0,1} (R_x(2\theta\alpha) \otimes I) C_{\text{not}0,1} C_{\text{not}1,0} (R_z(2\theta\beta) \otimes I) C_{\text{not}1,0}
$$

The steps of the solution can be found in the final of the notebook

In [3]:
def evolution(alpha, beta, t, n):
    """
    Simulates the quantum evolution of a two-qubit system under the influence of a Hamiltonian,
    H = α X⊗X + β Z⊗Z, using the Trotter-Suzuki decomposition.

    The Hamiltonian is decomposed into a sequence of quantum gates that approximate
    the unitary evolution operator U(t) = exp(-i H t) over a specified time interval `t`.

    Parameters:
    - alpha (float): Coefficient for the X⊗X term in the Hamiltonian.
    - beta (float): Coefficient for the Z⊗Z term in the Hamiltonian.
    - t (float): Total time of the evolution.
    - n (int): Number of Trotter steps.

    Returns:
    - np.ndarray: Probability distribution of the quantum state after evolution.

    The function uses Pennylane to create and execute a quantum circuit on the 'default.qubit'
    device. It returns the probability distribution of the state over the computational basis
    at the end of the evolution.

    The circuit consists of `n` repetitions of two sequences:
    1. A CNOT gate followed by an RX rotation and another CNOT gate to implement
       the unitary corresponding to exp(-i α θ X⊗X), where θ = t / n.
    2. A CNOT gate followed by an RZ rotation and another CNOT gate to implement
       the unitary corresponding to exp(-i β θ Z⊗Z).

    Example:
    >>> alpha = 0.5
    >>> beta = 0.3
    >>> t = 1.0
    >>> n = 10
    >>> probs = evolution(alpha, beta, t, n)
    >>> print(probs)
    """
    dev = qml.device('default.qubit', wires=2)
    @qml.qnode(dev)
    def circuit():
        theta = t / n
        for _ in range(n):
            # Implement exp(-i α θ X⊗X)
            qml.CNOT(wires=[0,1])
            qml.RX(2 * alpha * theta, wires=1)
            qml.CNOT(wires=[0,1])

            # Implement exp(-i β θ Z⊗Z)
            qml.CNOT(wires=[1,0])
            qml.RZ(2 * beta * theta, wires=0)
            qml.CNOT(wires=[1,0])

        return qml.probs(wires=[0,1])

    return circuit()


In [4]:
test_input =  [0.5,0.8,0.2,1]

evolution(0.5,0.8,0.2,1)

tensor([0.99003329, 0.00996671, 0.        , 0.        ], requires_grad=True)

My Hamiltonian is given by:

$$
H = \alpha X\otimes X + \beta Z \otimes Z
$$
(1)

From the Trotter relation we have:

$$
e^{A+B} = \lim_{n \to \infty} (e^{\frac{A}{n}} e^{\frac{B}{n}})^n
$$
(1.1)

For our case interest, we get:

$$
e^{-iHt} = \lim_{n \to \infty} (e^{\frac{-iHt}{n}})^n
$$
(1.2)

Where
$$
H_1 = \alpha X\otimes X \quad \text{and} \quad H_2 = \beta Z \otimes Z
$$

From the temporal evolution, we have:

$$
U(t) = e^{-iHt}
$$
(1.3)

In our case, we have:

$$
U(t) = e^{-i(H_1+H_2)t}
$$
(1.4)

Thus, we can rewrite (1.4) as:

$$
U(t) = \lim_{n \to \infty} \left(e^{-\frac{iH_1t}{n}} e^{-\frac{iH_2t}{n}}\right)^n
$$
(1.5)


To develop intuition, consdider the case of n=2. In this way, we have:
$$
\left(e^{iH_1t + iH_2t}\right)^2 = \left(e^{-\frac{iH_1t}{2}} e^{-\frac{iH_2t}{2}}\right)\left(e^{-\frac{iH_1t}{2}} e^{-\frac{iH_2t}{2}}\right)
$$
(1.6)

Considering an solution for finit n, we can rewrite the equation (1.5) as:
$$
U(t) \approx \prod_{i=1}^n e^{-iH_1\theta} e^{-iH_2\theta}
$$
(1.7)


Where  $$\theta = \frac{t}{n}$$. Furthermore, we can massage equation (1.7) and rewrite it in the following way:

$$
    U(t) \approx \prod_{i=1}^n \prod_{j=1}^2 e^{-iH_j\theta}
$$
(1.8)
However, we return to our interests. Note that:
$$
H_1 = \alpha X \otimes X,
$$
and can be written as:
$$
H_1 = \alpha C_{\text{not}0,1} (X \otimes I) C_{\text{not}0,1}
$$
(1.9)
in a similar way, we have :


$$
H_2 = \beta C_{\text{not}1,0} (Z \otimes I) C_{\text{not}1,0}
$$
(1.10)

Applying it in (1.8) eqaution we obtain the followiing expression:
$$
U(t) \approx \prod_{i=1}^n \exp\left(-i\alpha\theta C_{\text{not } 0,1} (X \otimes I) C_{\text{not } 0,1}\right)\exp\left(-i\beta\theta C_{\text{not } 1,0} (Z \otimes I) C_{\text{not } 1,0}\right)
$$
(1.11)


going further, we can use the following expression:
$$
e^{-i\theta U^\dagger A U} = U^\dagger e^{-i\theta A} U
$$
(1.12)

In this way, for \(H_1\) we have:
$$
\exp\left(-i\alpha\theta C_{\text{not } 0,1} (X \otimes I) C_{\text{not } 0,1}\right) = C_{\text{not } 0,1} \exp\left(-i\alpha\left(X \otimes I\right)\right) C_{\text{not } 0,1}
$$
(1.13)

that is, we have to:

$$
\exp\left(-i\alpha\theta C_{\text{not } 0,1} (X \otimes I) C_{\text{not } 0,1}\right) = C_{\text{not } 0,1}\left(R_x(2\theta\alpha)\right)C_{\text{not } 0,1}
$$

(1.14)
and of course, we have a similar term for $H_2$. So, in this way we can express the temporal evolution as:


$$
U(t) \approx \prod_{l=1}^n C_{\text{not}0,1} (R_x(2\theta\alpha) \otimes I) C_{\text{not}0,1} C_{\text{not}1,0} (R_z(2\theta\beta) \otimes I) C_{\text{not}1,0}
$$
(1.15)
